In [2]:
import pickle
import gzip
import numpy as np
import scipy
import pandas as pd
import keras
import keras.layers as layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.layers import Input, LSTM, Dense, Lambda, Conv1D, Conv2D, AveragePooling2D, AveragePooling1D, Flatten, MaxPooling2D, MaxPooling1D, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.utils import Sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras.applications import imagenet_utils
from keras import backend as K
from time import time
from keras import losses
from sklearn.metrics import  roc_curve, roc_auc_score, classification_report, confusion_matrix, accuracy_score, mean_absolute_error
import glob
from PIL import Image
import h5py
import random
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA, KernelPCA
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.gaussian_process as gp
import pandas as pd
import pickle
from joblib import dump, load
from sklearn.utils import shuffle
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
import warnings
warnings.filterwarnings('ignore')
!pip install silence_tensorflow

In [4]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import tensorflow as tf

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jan 15 20:53:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !rm -R qmc qmc1
    !git clone https://github.com/fagonzalezo/qmc.git
    !mv qmc qmc1
    !mv qmc1/qmc .
else:
    import sys
    sys.path.insert(0, "../")

Cloning into 'qmc'...
remote: Enumerating objects: 215, done.
remote: Total 215 (delta 0), reused 0 (delta 0), pack-reused 215
Receiving objects: 100% (215/215), 17.06 MiB | 26.84 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [8]:
import qmc.tf.layers as layers
import qmc.tf.models as models

In [ ]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((300,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  20.485220233073647 
    gamma_opt =  0.09305505964484745 
    learning_rate_opt =  0.0009492250482456459 
    alpha_opt =  0.6745501184141052 
    dim_x_opt =  266 
    num_eig_opt =  16

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((206,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(13, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=13, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
75/75 [==============================] - 2s 1ms/step
------------MAE = 0.3737864077669903
---------------------------------------- 2
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.3058252427184466
---------------------------------------- 3
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.3640776699029126
---------------------------------------- 4
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.3737864077669903
---------------------------------------- 5
75/75 [==============================] - 0s 950us/step
------------MAE = 0.34951456310679613
---------------------------------------- 6
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.35436893203883496
---------------------------------------- 7
75/75 [==============================] - 0s 959us/step
------------MAE = 0.3737864077669903
---------------------------------------- 8
75/75 [=============

In [ ]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((300,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  19.59225655911298 
    gamma_opt =  0.03730297717806879 
    learning_rate_opt =  0.0007359750748940179 
    alpha_opt =  0.11806777807443536 
    dim_x_opt =  648 
    num_eig_opt =  81

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((206,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(13, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=13, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
75/75 [==============================] - 2s 1ms/step
------------MAE = 0.2524271844660194
---------------------------------------- 2
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.2669902912621359
---------------------------------------- 3
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.3058252427184466
---------------------------------------- 4
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.32524271844660196
---------------------------------------- 5
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.2766990291262136
---------------------------------------- 6
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.2912621359223301
---------------------------------------- 7
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.3446601941747573
---------------------------------------- 8
75/75 [==================

In [ ]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((300,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt= 19.076864054645476 
    gamma_opt= 0.005947372436553702 
    learning_rate_opt= 0.0008934494477141322 
    alpha_opt= 0.22632790338813114 
    dim_x_opt= 1024 
    num_eig_opt= 32

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((206,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(13, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=13, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
75/75 [==============================] - 2s 3ms/step
------------MAE = 0.3300970873786408
---------------------------------------- 2
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.28640776699029125
---------------------------------------- 3
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.3737864077669903
---------------------------------------- 4
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.3155339805825243
---------------------------------------- 5
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.2669902912621359
---------------------------------------- 6
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.3592233009708738
---------------------------------------- 7
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.33980582524271846
---------------------------------------- 8
75/75 [=================

In [ ]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((300,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt= 4.305487914270835 
    gamma_opt= 0.007717095219618697 
    learning_rate_opt= 0.00043308858657742865 
    alpha_opt= 0.08294646708197106 
    dim_x_opt= 1024 
    num_eig_opt= 32

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((206,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(13, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=13, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.2815533980582524
---------------------------------------- 2
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.2669902912621359
---------------------------------------- 3
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.34951456310679613
---------------------------------------- 4
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.2815533980582524
---------------------------------------- 5
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.3106796116504854
---------------------------------------- 6
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.3058252427184466
---------------------------------------- 7
75/75 [==============================] - 0s 3ms/step
------------MAE = 0.41262135922330095
---------------------------------------- 8
75/75 [=================

In [ ]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((300,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt= 6.957615751003471 
    gamma_opt= 0.0021007332906673526 
    learning_rate_opt= 7.283463431708909e-05 
    alpha_opt= 0.02826020815242114 
    dim_x_opt= 256 
    num_eig_opt= 32

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((206,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(13, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=13, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
75/75 [==============================] - 0s 926us/step
------------MAE = 0.33495145631067963
---------------------------------------- 2
75/75 [==============================] - 0s 954us/step
------------MAE = 0.27184466019417475
---------------------------------------- 3
75/75 [==============================] - 0s 922us/step
------------MAE = 0.39805825242718446
---------------------------------------- 4
75/75 [==============================] - 0s 930us/step
------------MAE = 0.3058252427184466
---------------------------------------- 5
75/75 [==============================] - 0s 947us/step
------------MAE = 0.41262135922330095
---------------------------------------- 6
75/75 [==============================] - 0s 931us/step
------------MAE = 0.3640776699029126
---------------------------------------- 7
75/75 [==============================] - 0s 984us/step
------------MAE = 0.35436893203883496
---------------------------------------- 8
75/75 [

In [ ]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((300,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  19.852767357301687 
    gamma_opt =  0.02146822138483384 
    learning_rate_opt =  0.0003248581562407347 
    alpha_opt =  0.18840201883482777 
    dim_x_opt =  662 
    num_eig_opt =  82

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((206,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(13, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=13, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
75/75 [==============================] - 2s 1ms/step
------------MAE = 0.2815533980582524
---------------------------------------- 2
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.27184466019417475
---------------------------------------- 3
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.33495145631067963
---------------------------------------- 4
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.3058252427184466
---------------------------------------- 5
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.3446601941747573
---------------------------------------- 6
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.35436893203883496
---------------------------------------- 7
75/75 [==============================] - 0s 1ms/step
------------MAE = 0.3592233009708738
---------------------------------------- 8
75/75 [================

In [ ]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((300,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  4.707784687739816 
    gamma_opt =  0.013872331578599618 
    learning_rate_opt =  0.0008055495242510918 
    alpha_opt =  0.000687554630854903 
    dim_x_opt =  683 
    num_eig_opt =  170

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((206,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(13, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=13, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.24271844660194175
---------------------------------------- 2
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.2524271844660194
---------------------------------------- 3
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.33495145631067963
---------------------------------------- 4
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.22330097087378642
---------------------------------------- 5
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.32524271844660196
---------------------------------------- 6
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.32038834951456313
---------------------------------------- 7
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.30097087378640774
---------------------------------------- 8
75/75 [=============

In [9]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/boston/data/housing_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((300,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  17.801325158522143 
    gamma_opt =  0.03284642224423173 
    learning_rate_opt =  0.0006847387967772355 
    alpha_opt =  0.19172914845429934 
    dim_x_opt =  885 
    num_eig_opt =  221


    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((206,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(13, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=13, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
75/75 [==============================] - 1s 2ms/step
------------MAE = 0.2961165048543689
---------------------------------------- 2
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.2621359223300971
---------------------------------------- 3
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.32038834951456313
---------------------------------------- 4
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.30097087378640774
---------------------------------------- 5
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.28640776699029125
---------------------------------------- 6
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.3592233009708738
---------------------------------------- 7
75/75 [==============================] - 0s 2ms/step
------------MAE = 0.3300970873786408
---------------------------------------- 8
75/75 [================